# Calculate BPR coefficients for a street network

This notebook calculates the BPR (Bureau of Public Roads) curves' coefficients $a_0$, $a_1$, $a_2$, $a_3$, and $a_4$ for an OpenStreetMap street network of the bay area.

## BPR function definition:

$t_i = t0_i (1 + \alpha (\frac{v_i}{c_i}) ^ \beta)$

where:

$t_i =$ congested flow travel time on link $i$

$t0_i =$ free-flow travel time on link $i$

$v_i$ = volume of traffic on link $i$ per unit of time (flow attempting to use link $i$)

$c_i =$ capacity of link $i$ per unit of time

$\alpha =$ alpha coefficient, which was assigned a value of 0.15 in the original BPR curve

$\beta =$ beta coefficient, the exponent of the power function, which was assigned a value of 4 in the original BPR curve

We will use these default $\alpha$ and $\beta$ parameter values in this script, but note that they are not the only way to parameterize this function. For instance, TRB's *NHCRP Report 716: Travel Demand Forecasting Parameters and Techniques* pg 75 provides various coefficients estimated using the 1985 *Highway Capacity Manual*.


## BPR coefficients to be calculated in this script:

$a_0 = t0_i =$ the free-flow travel time

$a_1$, $a_2$, and $a_3$ are not used, they are always zero

$a_4 = t0_i \frac{\alpha}{c_i ^ {\beta}}$


## Our edge capacity and speed defaults, by highway type and number of lanes:

### Capacity defaults:

(in units of vehicles per lane per hour)

| OSM hwy type  | Description | lanes=1 | lanes=2 | lanes=3 | lanes=4+|
| ------------- | ------------- | ------------- | ------------- | ------------- | ------------- |
| motorway  | freeway, turnpike, or interstate  | 1900 | 2000 | 2000 | 2200 |
| trunk  | surface expressway, divided road, limited number of intersections  | 1900 | 2000 | 2000 | 2000 |
| primary  | arterial road  | 1000 | 1000 | 1000 | 1000 |
| secondary  | usually separated by a central line on road  | 900 | 900 | 900 | 900 |
| tertiary  | links local centers of activity with low to moderate traffic  | 900 | 900 | 900 | 900 |

### Max speed defaults:

(in units of miles per hour)

| OSM hwy type  | Description | lanes=1 | lanes=2 | lanes=3 | lanes=4+|
| ------------- | ------------- | ------------- | ------------- | ------------- | ------------- |
| motorway  | freeway, turnpike, or interstate  | 50 | 50 | 65 | 65 |
| trunk  | surface expressway, divided road, limited number of intersections  | 45 | 45 | 45 | 45 |
| primary  | arterial road  | 30 | 30 | 30 | 30 |
| secondary  | usually separated by a central line on road  | 25 |25 | 25 | 25 |
| tertiary  | links local centers of activity with low to moderate traffic  | 20 | 20 | 20 | 20 |

Note: these tables originally come from Maddie Sheehan's solver code for mapping OSM road types to road speeds/capacities.

In [1]:
import ast
import statistics
import pandas as pd
import time

start_time = time.time()

In [2]:
# define speed defaults for each hwy type and number of lanes, so we can infer when lacking data
speed_defaults = {'tertiary' : {1 : 20, 2 : 20, 3 : 20, 4 : 20, -1 : 20},
                  'tertiary_link' : {1 : 20, 2 : 20, 3 : 20, 4 : 20, -1 : 20},
                  'secondary' : {1 : 25, 2 : 25, 3 : 25, 4 : 25, -1 : 25},
                  'secondary_link' : {1 : 25, 2 : 25, 3 : 25, 4 : 25, -1 : 25},
                  'primary' : {1 : 30, 2 : 30, 3 : 30, 4 : 30, -1 : 30},
                  'primary_link' : {1 : 30, 2 : 30, 3 : 30, 4 : 30, -1 : 30},
                  'trunk' : {1 : 45, 2 : 45, 3 : 45, 4 : 45, -1 : 45},
                  'trunk_link' : {1 : 45, 2 : 45, 3 : 45, 4 : 45, -1 : 45},
                  'motorway' : {1 : 50, 2 : 50, 3 : 65, 4 : 65, -1 : 57.5},
                  'motorway_link' : {1 : 50, 2 : 50, 3 : 65, 4 : 65, -1 : 57.5},
                  'unclassified' : {1 : 20, 2 : 20, 3 : 20, 4 : 20, -1 : 20},
                  'road' : {1 : 30, 2 : 30, 3 : 30, 4 : 30, -1 : 30}}

# define per-lane capacity defaults for each hwy type and number of lanes, so we can infer when lacking data
capacity_defaults = {'tertiary' : {1 : 900, 2 : 900, 3 : 900, 4 : 900, -1 : 900},
                     'tertiary_link' : {1 : 900, 2 : 900, 3 : 900, 4 : 900, -1 : 900},
                     'secondary' : {1 : 900, 2 : 900, 3 : 900, 4 : 900, -1 : 900},
                     'secondary_link' : {1 : 900, 2 : 900, 3 : 900, 4 : 900, -1 : 900},
                     'primary' : {1 : 1000, 2 : 1000, 3 : 1000, 4 : 1000, -1 : 1000},
                     'primary_link' : {1 : 1000, 2 : 1000, 3 : 1000, 4 : 1000, -1 : 1000},
                     'trunk' : {1 : 1900, 2 : 2000, 3 : 2000, 4 : 2000, -1 : 1975},
                     'trunk_link' : {1 : 1900, 2 : 2000, 3 : 2000, 4 : 2000, -1 : 1975},
                     'motorway' : {1 : 1900, 2 : 2000, 3 : 2000, 4 : 2200, -1 : 2025},
                     'motorway_link' : {1 : 1900, 2 : 2000, 3 : 2000, 4 : 2200, -1 : 2025},
                     'unclassified' : {1 : 800, 2 : 800, 3 : 800, 4 : 800, -1 : 800},
                     'road' : {1 : 900, 2 : 900, 3 : 900, 4 : 900, -1 : 900}}

# note: -1 is the key for the null value
# note: highway_links are given the same values as their highway types
# note: 'road' is effectively an OSM null highway type
# note: 'unclassified' is a highway type one step below tertiary in the OSM hierarchy

## Load the data

In [3]:
# load the edge data
edges_all = pd.read_csv('data/bay_area_edges.csv', low_memory=False)
len(edges_all)

66112

In [4]:
# retain only useful columns
useful_cols = ['uniqueid', 'u', 'v', 'length', 'maxspeed', 'lanes', 'highway']
ed = edges_all[useful_cols].copy()
ed.count()

uniqueid    66112
u           66112
v           66112
length      66112
maxspeed    17909
lanes       28768
highway     66112
dtype: int64

## Process `highway` type values

In [5]:
# convert string representations of lists to lists
def convert_lists(value):
    if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
        return ast.literal_eval(value) #parse str -> list
    else:
        return value

In [6]:
# convert string representation of multiple highway types to a list
ed['highway'] = ed['highway'].map(convert_lists)

In [7]:
# collapse multiple highway type values into a single value
def collapse_multiple_hwy_values(hwy):
    if isinstance(hwy, list):
        # if we find an item in our defaults dict, use that value
        # otherwise, just use the zeroth item in the list
        for item in hwy:
            if item in speed_defaults.keys():
                return item
        return hwy[0]
    else:
        return hwy

ed['highway'] = ed['highway'].map(collapse_multiple_hwy_values)

In [8]:
ed['highway'].value_counts().sort_index()

motorway           2692
motorway_link      4900
primary            8652
primary_link       1208
road                172
secondary         17468
secondary_link     1116
tertiary          18787
tertiary_link       547
trunk              1989
trunk_link          980
unclassified       7601
Name: highway, dtype: int64

## Process number of `lanes` values

In [9]:
# convert string representation of multiple lane values to a list
ed['lanes'] = ed['lanes'].map(convert_lists)

In [10]:
# collapse multiple lane values into a single value
def collapse_multiple_lane_values(value):
    if isinstance(value, list):
        # return the mean of the values in the list
        numeric_values = [int(x) for x in value]
        return int(statistics.mean(numeric_values))
    else:
        return value

ed['lanes'] = ed['lanes'].map(collapse_multiple_lane_values)

In [11]:
# calculate typical number of lanes per hwy type
ed['lanes'] = ed['lanes'].astype(float)
lane_defaults = ed.groupby('highway')['lanes'].median()
lane_defaults = lane_defaults.fillna(value=2).to_dict() #'road' type is null
lane_defaults

{'motorway': 4.0,
 'motorway_link': 1.0,
 'primary': 3.0,
 'primary_link': 1.0,
 'road': 1.0,
 'secondary': 3.0,
 'secondary_link': 1.0,
 'tertiary': 2.0,
 'tertiary_link': 1.0,
 'trunk': 3.0,
 'trunk_link': 1.0,
 'unclassified': 2.0}

In [12]:
# impute number of lanes when data is missing
def impute_lanes(row):
    if pd.notnull(row['lanes']):
        return row['lanes']
    else:
        return lane_defaults[row['highway']]

ed['lanes'] = ed.apply(impute_lanes, axis=1).astype(int)

In [13]:
ed['lanes'].value_counts().sort_index()

1      8399
2     31721
3     19254
4      4577
5      1795
6       293
7        61
8        11
11        1
Name: lanes, dtype: int64

In [14]:
# make 4 lanes the capped value (for 4+ lanes dict lookup), but retain true lanes value in lanes column
ed['lanes_capped'] = ed['lanes']
ed.loc[ed['lanes_capped'] > 4, 'lanes_capped'] = 4

In [15]:
ed['lanes_capped'].value_counts().sort_index()

1     8399
2    31721
3    19254
4     6738
Name: lanes_capped, dtype: int64

## Process `maxspeed` values

In [16]:
# convert string representation of multiple maxspeed values to a list
ed['maxspeed'] = ed['maxspeed'].map(convert_lists)

In [17]:
# collapse multiple maxspeed values into a single value
def collapse_multiple_maxspeed_values(value):
    if isinstance(value, list):
        try:
            # strip non-numeric " mph" from each value in the list then take the mean
            values = [int(x.replace(' mph', '')) for x in value]
            return statistics.mean(values)
        except:
            # if exception, return null (a few have multiple values like "35 mph;40 mph")
            return None
    else:
        return value

ed['maxspeed'] = ed['maxspeed'].map(collapse_multiple_maxspeed_values)

In [18]:
# for all string maxspeed values, strip non-numeric " mph" from each value
def parse_speed_strings(value):
    if isinstance(value, str):
        value = value.replace(' mph', '')
        # sometimes multiple speeds are semicolon-delimited -- collapse to a single value
        if ';' in value:
            # return the mean of the values if it has that semicolon
            values = [int(x) for x in value.split(';')]
            return statistics.mean(values)
        else:
            return int(value)
    else:
        return value
    
ed['maxspeed'] = ed['maxspeed'].map(parse_speed_strings)

In [19]:
ed['maxspeed'].value_counts().sort_index()

10.000000      17
15.000000      40
17.500000       4
20.000000      45
22.500000       8
25.000000    3757
26.666667       2
27.500000      46
30.000000    2160
32.500000      80
35.000000    4518
36.250000       4
37.500000      90
38.333333       6
40.000000    1967
41.000000       2
41.250000       2
41.666667       2
42.500000      33
43.333333       2
44.000000       2
45.000000    1784
47.500000      29
50.000000     759
52.500000      23
55.000000     449
56.333333       2
56.666667       4
57.500000       4
60.000000      18
65.000000    1991
67.500000       2
70.000000      44
80.000000       7
Name: maxspeed, dtype: int64

## Calculate free-flow speed for each edge

In [20]:
# infer speed from defaults based on highway type classifier and number of lanes
def infer_speed(row):
    hwy = row['highway']
    lanes = row['lanes_capped']
    return speed_defaults[hwy][lanes]

In [21]:
# extract maxspeed from OSM data when it already exists
known_speeds = ed[pd.notnull(ed['maxspeed'])]['maxspeed']
known_speeds = known_speeds.astype(int)

In [22]:
# infer speed on all other edges that lack maxspeed data
inferred_speeds = ed[pd.isnull(ed['maxspeed'])].apply(infer_speed, axis=1)

# merge known speeds with inferred speeds to get a free-flow speed for each edge
ed['speed_mph'] = known_speeds.append(inferred_speeds, ignore_index=False, verify_integrity=True)

## Infer capacity for each edge

In [23]:
# infer capacity per lane per hour from defaults based on highway type classifier and number of lanes
def infer_capacity(row):
    hwy = row['highway']
    lanes = row['lanes_capped']
    return capacity_defaults[hwy][lanes]

In [24]:
# infer per-lane capacity for each edge using capacity defaults
ed['capacity_lane_hour'] = ed.apply(infer_capacity, axis=1)

In [25]:
# convert capacity per lane per hour to capacity per edge per hour
ed['capacity_hour'] = ed['capacity_lane_hour'] * ed['lanes']

In [26]:
ed.head()

,uniqueid,u,v,length,maxspeed,lanes,highway,lanes_capped,speed_mph,capacity_lane_hour,capacity_hour
0,0,814743619,91052181,2005.778127,NaN,2,tertiary,2,20,900,1800
1,1,309330024,253297204,320.848230,NaN,5,motorway,4,65,2200,11000
2,2,4353687911,4353687912,9.930174,NaN,1,secondary_link,1,25,900,900
3,3,4353687911,4353689485,408.612119,30.0,2,secondary,2,30,900,1800
4,4,4353687912,4353687911,9.930174,NaN,1,secondary_link,1,25,900,900


## Calculate BPR coefficients

In [27]:
# define the parameter values (see documentation at top)
alpha = 0.15
beta = 4

We want travel times in seconds, but our speeds are in miles per hour and our edge lengths are in meters, so we need to do some conversion. Also, we want capacities in vehicles per edge per second, but they are currently in units of vehicles per edge per hour, so we need to convert this.

In [28]:
meters_per_mile = 1609.34 #factor to convert meters to miles
seconds_per_hour = 60 * 60 #factor to convert hours to seconds

In [29]:
# calculate a0 as free-flow travel time for each edge, in seconds
ed['length_miles'] = ed['length'] / meters_per_mile
ed['a0'] = ed['length_miles'] / ed['speed_mph']
ed['a0'] = ed['a0'] * seconds_per_hour

In [30]:
# a1, a2, and a3 are not used, they are always zero
ed['a1'] = 0
ed['a2'] = 0
ed['a3'] = 0

In [31]:
# convert capacity per hour to capacity per second
ed['capacity'] = ed['capacity_hour'] / seconds_per_hour

In [32]:
# calculate a4 = a0 * alpha / (capacity ^ beta)
ed['a4'] = ed['a0'] * alpha / (ed['capacity'] ** beta)

## Save to disk

In [33]:
# reformat data into format LBNL desires
save_cols = ['uniqueid', 'u', 'v', 'a0', 'a1', 'a2', 'a3', 'a4']
ed_save = ed[save_cols]
ed_save = ed_save.rename(columns={'uniqueid':'LINK', 'u':'A', 'v':'B'})
ed_save.head(10)

,LINK,A,B,a0,a1,a2,a3,a4
0,0,814743619,91052181,224.340452,0,0,0,538.417084
1,1,309330024,253297204,11.041828,0,0,0,0.019001
2,2,4353687911,4353687912,0.888529,0,0,0,34.119507
3,3,4353687911,4353689485,30.468052,0,0,0,73.123324
4,4,4353687912,4353687911,0.888529,0,0,0,34.119507
5,5,4353687912,65314953,22.676780,0,0,0,54.424271
6,6,386924997,4906347242,2.561638,0,0,0,0.252104
7,7,386924997,386925477,6.611620,0,0,0,0.650683
8,8,386924997,1700584688,19.589130,0,0,0,37.870584
9,9,354419176,354413173,87.140706,0,0,0,1.694015


In [34]:
# save as csv file
ed_save.to_csv('data/bay_area_bpr.csv', index=False, encoding='utf-8')

In [35]:
# process is all done, show elapsed time
elapsed_time = time.time() - start_time
elapsed_time

5.821817398071289